In [1]:
# docker build -t private_mirror .
# docker run --rm private_mirror verify_file silicon tests/local/evaluate/test_constructor.py

# where "private_mirror" is the image tag

In [2]:
import os
from pathlib import Path
import subprocess
from tqdm import tqdm
import pandas as pd
from dataclasses import dataclass
from statistics import mean
import logging
import datetime

from time import sleep
import random

pd.set_option('display.precision', 2)

SHOWCASE = ("simple_parent", "wrapper", "prefix", "lock", "getattribute_and_getattr")

In [3]:
logger = logging.getLogger(__name__)
logging.basicConfig(filename=str(datetime.datetime.now()).replace(' ', '_'), encoding='utf-8', level=logging.DEBUG)

In [4]:
# Data = namedtuple('Data', ['name', 'lines_of_code', 'runs_silicon', 'runs_carbon'])
@dataclass
class Data:
    name: str
    lines_of_code_simple: int = -1
    lines_of_code_complex: int = -1
    runs_silicon_simple: list = None
    runs_silicon_complex: list = None
    runs_carbon_simple: list = None
    runs_carbon_complex: list = None

In [5]:
evaluate_path_from_notebook = Path('../tests/local/evaluate/')
evaluate_path_inside_image = Path('tests/local/evaluate/')
showcase_path_from_notebook = Path('../tests/local/showcase/')
showcase_path_inside_image = Path('tests/local/showcase/')

evaluate_files = [showcase_path_inside_image / f for f in os.listdir(showcase_path_from_notebook) if not f.startswith('.')]
evaluate_files += sorted([evaluate_path_inside_image / f for f in os.listdir(evaluate_path_from_notebook) if not f.startswith('.')], key=lambda x: x.stem)

results = {f.stem: Data(f.stem) for f in evaluate_files if not f.stem.endswith('_complex')}

In [6]:
pbar = tqdm(evaluate_files)
for f in pbar:

    runs = {"silicon": [], "carbon": []}
    lines_of_code = 0
    for verifier in runs.keys():
        for i in range(3):
            pbar.set_description(f"procesing {f.stem} {verifier} {i}")
            logger.info(f"procesing {f.stem} {verifier} {i}")
            command = ["docker", "run", "--rm", "private_mirror", "evaluate_file", verifier, str(f)]

            res = subprocess.run(command, capture_output = True, text=True)
            logger.info(res)
            res = res.stdout.split('\n')
            lines = int(res[-2].split(" ")[0])
            time = float(res[-3].split(" ")[2])

            runs[verifier].append(time)
            lines_of_code = lines

    if f.stem.endswith('_complex'):
        stem = f.stem[:-8]
        results[stem].lines_of_code_complex = lines_of_code
        results[stem].runs_silicon_complex = runs["silicon"]
        results[stem].runs_carbon_complex = runs["carbon"]
    elif f.stem in (SHOWCASE):
        results[f.stem].lines_of_code_complex = lines_of_code
        results[f.stem].runs_silicon_complex = runs["silicon"]
        results[f.stem].runs_carbon_complex = runs["carbon"]
    else:
        results[f.stem].lines_of_code_simple = lines_of_code
        results[f.stem].runs_silicon_simple = runs["silicon"]
        results[f.stem].runs_carbon_simple = runs["carbon"]

procesing test_student_enroll_preds_complex carbon 2: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [47:20<00:00, 105.22s/it]


In [7]:
df_data = [d.__dict__ for d in results.values()]
df = pd.DataFrame(df_data)
df['avg_silicon_simple'] = df.apply(lambda x: mean(x['runs_silicon_simple']) if x['runs_silicon_simple'] else "", axis=1)
df['avg_silicon_complex'] = df.apply(lambda x: mean(x['runs_silicon_complex']) if x['runs_silicon_complex'] else "", axis=1)
df['avg_carbon_simple'] = df.apply(lambda x: mean(x['runs_carbon_simple']) if x['runs_carbon_simple'] else "", axis=1)
df['avg_carbon_complex'] = df.apply(lambda x: mean(x['runs_carbon_complex']) if x['runs_carbon_complex'] else "", axis=1)
df.replace(to_replace=[None, -1], value="", inplace=True)
df

,name,lines_of_code_simple,lines_of_code_complex,runs_silicon_simple,runs_silicon_complex,runs_carbon_simple,runs_carbon_complex,avg_silicon_simple,avg_silicon_complex,avg_carbon_simple,avg_carbon_complex
0,simple_parent,,1164,,"[11.35, 10.24, 10.0]",,"[9.88, 9.21, 9.29]",,10.53,,9.46
1,getattribute_and_getattr,,1236,,"[11.17, 11.2, 11.62]",,"[11.13, 11.77, 11.4]",,11.33,,11.43
2,lock,,1240,,"[11.65, 11.38, 11.95]",,"[11.59, 11.28, 11.29]",,11.66,,11.39
3,prefix,,1159,,"[9.97, 10.33, 9.96]",,"[9.56, 9.59, 9.6]",,10.09,,9.58
4,wrapper,,1235,,"[10.59, 10.61, 10.73]",,"[10.01, 10.07, 9.96]",,10.64,,10.01
5,cav_example,1630,1726,"[16.24, 16.34, 16.32]","[20.71, 20.8, 22.16]","[21.38, 18.56, 18.45]","[30.75, 30.21, 30.45]",16.3,21.22,19.46,30.47
6,iap_bst,1878,2071,"[15.37, 15.32, 15.07]","[98.38, 75.01, 96.98]","[20.48, 20.32, 21.55]","[86.58, 84.46, 84.86]",15.25,90.12,20.78,85.30
7,keon_knapsack,1822,1945,"[17.3, 17.75, 17.85]","[41.78, 36.08, 36.98]","[27.05, 30.24, 25.09]","[44.7, 36.21, 43.8]",17.63,38.28,27.46,41.57
8,parkinson_recell,1268,1356,"[9.42, 9.63, 9.58]","[11.12, 11.44, 11.08]","[9.32, 9.47, 9.25]","[10.45, 10.81, 10.76]",9.54,11.21,9.35,10.67
9,test_constructor,1164,1287,"[9.57, 9.89, 9.58]","[13.21, 13.17, 13.38]","[9.44, 9.34, 9.49]","[10.57, 10.67, 10.53]",9.68,13.25,9.42,10.59


In [8]:
df.to_pickle("./evaluation.pkl") 

In [11]:
for _, r in df.iterrows():
    print(f"{str(r['name']).replace('_', '\\_')} & {r.lines_of_code_simple if r.lines_of_code_simple != -1 else ""} & {r.lines_of_code_complex} & ", end="")

    if r.avg_silicon_simple:
        print(f"{r.avg_silicon_simple:.2f} & ", end="")
    else:
        print(" & ", end="")

    if r.avg_silicon_complex:
        print(f"{r.avg_silicon_complex:.2f} & ", end="")
    else:
        print(" & ", end="")

    if r.avg_carbon_simple:
        print(f"{r.avg_carbon_simple:.2f} & ", end="")
    else:
        print(" & ", end="")

    if r.avg_carbon_complex:
        print(f"{r.avg_carbon_complex:.2f} \\\\ ", end="")
    else:
        print(" \\\\ ", end="")
    print()

simple\_parent &  & 1164 &  & 10.53 &  & 9.46 \\ 
getattribute\_and\_getattr &  & 1236 &  & 11.33 &  & 11.43 \\ 
lock &  & 1240 &  & 11.66 &  & 11.39 \\ 
prefix &  & 1159 &  & 10.09 &  & 9.58 \\ 
wrapper &  & 1235 &  & 10.64 &  & 10.01 \\ 
cav\_example & 1630 & 1726 & 16.30 & 21.22 & 19.46 & 30.47 \\ 
iap\_bst & 1878 & 2071 & 15.25 & 90.12 & 20.78 & 85.30 \\ 
keon\_knapsack & 1822 & 1945 & 17.63 & 38.28 & 27.46 & 41.57 \\ 
parkinson\_recell & 1268 & 1356 & 9.54 & 11.21 & 9.35 & 10.67 \\ 
test\_constructor & 1164 & 1287 & 9.68 & 13.25 & 9.42 & 10.59 \\ 
test\_dynamic\_field\_creation & 1856 & 1933 & 10.12 & 11.92 & 10.72 & 11.65 \\ 
test\_havoced\_types & 1768 & 1900 & 12.18 & 13.97 & 12.53 & 12.96 \\ 
test\_isinstance & 981 & 1085 & 9.01 & 9.90 & 8.46 & 9.10 \\ 
test\_operators & 1783 & 1893 & 11.08 & 12.88 & 10.89 & 11.71 \\ 
test\_predicate & 1254 & 1370 & 9.60 & 11.91 & 9.64 & 10.72 \\ 
test\_student\_enroll\_preds & 1781 & 1901 & 12.79 & 14.61 & 13.04 & 14.74 \\ 
